In [ ]:
# ==============================================================================
# ROBÔ PROJUDI - COMPLETO COM DOWNLOAD DE PDFs E SALVAMENTO DE HTMLs NO BANCO
# ==============================================================================

import pandas as pd
import unicodedata
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time
import logging
import sqlite3
import os
import random
from webdriver_manager.chrome import ChromeDriverManager
from colorama import init, Fore, Style

# ==============================================================================
# CONFIGURAÇÃO
# ==============================================================================

init(autoreset=True)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(threadName)s - %(levelname)s - %(message)s')

NOME_BANCO_DADOS = 'processos_v2.db'
BANCO_ESTRATEGICO = 'precatorios_estrategico.db'
ARQUIVO_PROCESSOS_CSV = 'processos_devedores.csv'
PASTA_DOWNLOADS = os.path.abspath("downloads_projudi")

POOL_DE_CREDENCIAIS = [
    {"usuario": "85413054149", "senha": "Cioni6725-"},
    {"usuario": "07871865625", "senha": "asdASD00-"}
]

# Configuração de filtros (será preenchida no início)
CONFIG = {
    "palavras_chave": [],  # Palavras positivas
    "palavras_nega": []    # Palavras negativas
}

# ==============================================================================
# FUNÇÃO DE FORMATAÇÃO CNJ
# ==============================================================================

def formatar_numero_cnj(numero):
    """Formata número de processo para padrão CNJ"""
    apenas_digitos = ''.join(filter(str.isdigit, numero))
    if len(apenas_digitos) != 20:
        return numero
    return f"{apenas_digitos[0:7]}-{apenas_digitos[7:9]}.{apenas_digitos[9:13]}.{apenas_digitos[13]}.{apenas_digitos[14:16]}.{apenas_digitos[16:20]}"

# ==============================================================================
# FUNÇÕES DO BANCO ESTRATÉGICO
# ==============================================================================

def adicionar_coluna_status_se_necessario():
    """Adiciona colunas de status na tabela ExtracaoDiario"""
    with sqlite3.connect(BANCO_ESTRATEGICO) as conn:
        cursor = conn.cursor()
        try:
            cursor.execute("ALTER TABLE ExtracaoDiario ADD COLUMN status_coleta TEXT DEFAULT 'pendente'")
            cursor.execute("ALTER TABLE ExtracaoDiario ADD COLUMN data_ultima_tentativa TEXT")
            cursor.execute("ALTER TABLE ExtracaoDiario ADD COLUMN mensagem_erro TEXT")
            conn.commit()
        except sqlite3.OperationalError:
            pass  # Colunas já existem

def buscar_processos_do_banco(quantidade):
    """Busca processos pendentes do banco"""
    with sqlite3.connect(BANCO_ESTRATEGICO) as conn:
        cursor = conn.cursor()
        cursor.execute("""
            SELECT id, numero_processo_cnj 
            FROM ExtracaoDiario 
            WHERE status_coleta IS NULL OR status_coleta IN ('pendente', 'erro')
            LIMIT ?
        """, (quantidade,))
        return cursor.fetchall()

def atualizar_status_processo(processo_id, status, mensagem_erro=None):
    """Atualiza status do processo"""
    from datetime import datetime
    with sqlite3.connect(BANCO_ESTRATEGICO) as conn:
        cursor = conn.cursor()
        cursor.execute("""
            UPDATE ExtracaoDiario 
            SET status_coleta = ?, data_ultima_tentativa = ?, mensagem_erro = ?
            WHERE id = ?
        """, (status, datetime.now().isoformat(), mensagem_erro, processo_id))
        conn.commit()

# ==============================================================================
# FUNÇÕES DO BANCO DE DADOS
# ==============================================================================

def salvar_dados_completos(numero_processo, dados_gerais, movimentacoes, decisoes_html):
    """Salva dados do processo, movimentações e decisões HTML no banco"""
    with sqlite3.connect(NOME_BANCO_DADOS) as conn:
        cursor = conn.cursor()
        
        # Garante que o processo exista
        cursor.execute("SELECT id FROM Processos WHERE numero_processo = ?", (numero_processo,))
        resultado = cursor.fetchone()
        if resultado:
            processo_id = resultado[0]
        else:
            cursor.execute("INSERT INTO Processos (numero_processo) VALUES (?)", (numero_processo,))
            processo_id = cursor.lastrowid

        # Atualiza dados gerais do processo
        cursor.execute("""UPDATE Processos 
                         SET data_distribuicao = ?, valor_causa = ?, data_ultima_coleta = CURRENT_TIMESTAMP 
                         WHERE id = ?""", 
                       (dados_gerais.get('data_distribuicao'), dados_gerais.get('valor_causa'), processo_id))

        # Histórico de estado
        cursor.execute("""SELECT classe_judicial, assunto, fase_processual 
                         FROM HistoricoEstado 
                         WHERE processo_id = ? 
                         ORDER BY data_coleta DESC LIMIT 1""", (processo_id,))
        ultimo_estado = cursor.fetchone()
        estado_atual = (
            dados_gerais.get('classe_judicial', 'N/A'), 
            dados_gerais.get('assunto', 'N/A'), 
            dados_gerais.get('fase_processual', 'N/A')
        )
        
        if not ultimo_estado or ultimo_estado != estado_atual:
            cursor.execute("""INSERT INTO HistoricoEstado 
                             (processo_id, classe_judicial, assunto, fase_processual) 
                             VALUES (?, ?, ?, ?)""", (processo_id, *estado_atual))

        # Limpa movimentações antigas e insere novas
        cursor.execute("DELETE FROM Movimentacoes WHERE processo_id = ?", (processo_id,))
        
        if movimentacoes:
            for mov_id_projudi, data_mov, desc, usr in movimentacoes:
                cursor.execute("""INSERT INTO Movimentacoes 
                                 (processo_id, data_movimentacao, descricao, usuario, movimentacao_id_projudi) 
                                 VALUES (?, ?, ?, ?, ?)""", 
                               (processo_id, data_mov, desc, usr, mov_id_projudi))
                
                # Se houver decisão HTML para esta movimentação, salva
                if mov_id_projudi in decisoes_html:
                    movimentacao_db_id = cursor.lastrowid
                    texto_decisao = decisoes_html[mov_id_projudi]
                    cursor.execute("INSERT INTO Decisoes (movimentacao_id, texto_completo) VALUES (?, ?)", 
                                  (movimentacao_db_id, texto_decisao))
        
        conn.commit()
        logging.info(f"✅ SUCESSO: {len(movimentacoes)} movimentações e {len(decisoes_html)} decisões HTML salvas para {numero_processo}")

# ==============================================================================
# FUNÇÕES DE EXTRAÇÃO
# ==============================================================================

def remover_acentos(texto):
    """Remove acentos de um texto para comparação flexível"""
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )

def filtrar_movimentacao(descricao_movimentacao):
    """Filtra movimentação baseado em palavras-chave positivas e negativas na DESCRIÇÃO"""
    if not CONFIG["palavras_chave"] and not CONFIG["palavras_nega"]:
        return True  # Sem filtros, aceita tudo
    
    # Para palavras POSITIVAS: remove acentos e busca substring (máxima flexibilidade)
    desc_sem_acento = remover_acentos(descricao_movimentacao.lower())
    
    # Se tem palavras positivas, precisa ter pelo menos uma (busca flexível)
    if CONFIG["palavras_chave"]:
        tem_positiva = any(
            remover_acentos(palavra.lower()) in desc_sem_acento 
            for palavra in CONFIG["palavras_chave"]
        )
        if not tem_positiva:
            return False
    
    # Para palavras NEGATIVAS: busca exata com acentos (menos flexível)
    desc_lower = descricao_movimentacao.lower()
    if CONFIG["palavras_nega"]:
        tem_negativa = any(palavra.lower() in desc_lower for palavra in CONFIG["palavras_nega"])
        if tem_negativa:
            return False
    
    return True

def extrair_dados_gerais(soup):
    """Extrai dados gerais do cabeçalho do processo"""
    dados_gerais = {}
    
    def find_value_by_label(label_text):
        try:
            label_tag = soup.find(lambda tag: tag.name == 'div' and tag.get_text(strip=True) == label_text)
            if label_tag and (value_tag := label_tag.find_next_sibling('span')):
                return ' '.join(value_tag.get_text(strip=True).split())
            return "N/A"
        except Exception:
            return "N/A"
    
    dados_gerais.update({
        'classe_judicial': find_value_by_label('Classe'),
        'assunto': find_value_by_label('Assunto(s)'),
        'fase_processual': find_value_by_label('Fase Processual'),
        'data_distribuicao': find_value_by_label('Dt. Distribuição'),
        'valor_causa': find_value_by_label('Valor da Causa')
    })
    
    return dados_gerais

def extrair_decisoes_publicas(driver, numero_processo):
    """
    Extrai TODAS as decisões HTML públicas ANTES de pedir acesso
    Retorna: dicionário {mov_id: texto_decisao}
    """
    logging.info(f"📝 Coletando decisões públicas (antes de pedir acesso)...")
    
    decisoes_html = {}
    aba_principal = driver.current_window_handle
    
    try:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Busca movimentações com destaque (decisões, despachos, atos judiciais)
        linhas_mov = soup.find_all('tr', class_='filtro-entrada')
        linhas_com_decisao = [linha for linha in linhas_mov if 'movi-destaque' in linha.get('class', [])]
        
        logging.info(f"📋 Encontradas {len(linhas_com_decisao)} movimentações com possíveis decisões")
        
        for idx, mov_tr in enumerate(linhas_com_decisao, 1):
            try:
                # Extrai ID da movimentação
                mov_id_projudi_tag = mov_tr.find('div', class_='dropMovimentacao')
                mov_id_projudi = mov_id_projudi_tag['id_movi'] if mov_id_projudi_tag else None
                
                if not mov_id_projudi:
                    continue
                
                # Tenta expandir a movimentação
                expand_link = mov_tr.find('img', id=lambda x: x and x.startswith('MostrarArquivos_'))
                if not expand_link:
                    continue
                
                # Clica para expandir
                ActionChains(driver).move_to_element(
                    driver.find_element(By.ID, expand_link['id'])
                ).click().perform()
                time.sleep(0.5)  # Reduzido de 1s para 0.5s
                
                # Atualiza soup
                soup_atualizado = BeautifulSoup(driver.page_source, 'html.parser')
                linha_arquivos = soup_atualizado.find('img', id=expand_link['id']).find_parent('tr').find_next_sibling('tr')
                
                if not linha_arquivos:
                    # Recolhe
                    ActionChains(driver).move_to_element(
                        driver.find_element(By.ID, expand_link['id'])
                    ).click().perform()
                    time.sleep(0.3)  # Reduzido de 0.5s para 0.3s
                    continue
                
                # Procura link HTML (decisão)
                links_arquivos = linha_arquivos.find_all('a', href=True)
                
                for link in links_arquivos:
                    href = link['href']
                    
                    if 'BuscaProcesso?PaginaAtual=6' in href:
                        # É HTML (decisão) - extrai texto
                        logging.info(f"📝 Extraindo decisão pública {idx}/{len(linhas_com_decisao)}: mov {mov_id_projudi}")
                        try:
                            driver.execute_script(f"window.open('{href}', '_blank');")
                            time.sleep(1)  # Reduzido de 2s para 1s
                            
                            nova_aba = [aba for aba in driver.window_handles if aba != aba_principal][0]
                            driver.switch_to.window(nova_aba)
                            
                            soup_decisao = BeautifulSoup(driver.page_source, 'html.parser')
                            texto_decisao = soup_decisao.get_text(separator='\n', strip=True)
                            decisoes_html[mov_id_projudi] = texto_decisao
                            
                            driver.close()
                            driver.switch_to.window(aba_principal)
                            logging.info(f"✅ Decisão pública extraída e salva")
                        except Exception as e:
                            logging.error(f"❌ Erro ao extrair decisão pública: {e}")
                            if len(driver.window_handles) > 1:
                                driver.switch_to.window(aba_principal)
                
                # Recolhe a movimentação
                ActionChains(driver).move_to_element(
                    driver.find_element(By.ID, expand_link['id'])
                ).click().perform()
                time.sleep(0.3)  # Reduzido de 0.5s para 0.3s
                
            except Exception as e:
                logging.warning(f"⚠️ Erro ao processar movimentação {idx}: {e}")
                continue
        
        logging.info(f"✅ Coleta pública concluída: {len(decisoes_html)} decisões extraídas")
        return decisoes_html
        
    except Exception as e:
        logging.error(f"❌ Erro na coleta de decisões públicas: {e}")
        return decisoes_html

def processar_movimentacoes_e_arquivos(driver, wait, numero_processo, decisoes_publicas):
    """Processa movimentações, baixa PDFs e extrai HTMLs (após obter acesso)"""
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    movimentacoes = []
    decisoes_html = dict(decisoes_publicas)  # Copia as decisões públicas já coletadas
    pdfs_baixados = 0
    htmls_novos = 0
    aba_principal = driver.current_window_handle
    
    linhas_mov = soup.find_all('tr', class_='filtro-entrada')
    logging.info(f"📋 Processando {len(linhas_mov)} movimentações (com acesso)")
    
    for idx, mov_tr in enumerate(linhas_mov, 1):
        colunas = mov_tr.find_all('td')
        if len(colunas) < 4:
            continue

        # Extrai dados da movimentação
        descricao_completa = colunas[1].get_text(separator=' ', strip=True)
        data_mov = colunas[2].text.strip()
        usuario_mov = colunas[3].text.strip()
        
        mov_id_projudi_tag = mov_tr.find('div', class_='dropMovimentacao')
        mov_id_projudi = mov_id_projudi_tag['id_movi'] if mov_id_projudi_tag else None
        
        movimentacoes.append((mov_id_projudi, data_mov, descricao_completa, usuario_mov))

        # Aplica filtro na DESCRIÇÃO da movimentação
        if not filtrar_movimentacao(descricao_completa):
            logging.info(f"⏭️ Movimentação {idx} ignorada pelo filtro (descrição não corresponde)")
            continue

        # Processa arquivos anexados (PDFs e HTMLs)
        try:
            # Tenta expandir a movimentação
            expand_link = mov_tr.find('img', id=lambda x: x and x.startswith('MostrarArquivos_'))
            if not expand_link:
                continue
            
            # Clica para expandir
            ActionChains(driver).move_to_element(
                driver.find_element(By.ID, expand_link['id'])
            ).click().perform()
            time.sleep(0.5)  # Reduzido de 1s para 0.5s
            
            # Atualiza soup
            soup_atualizado = BeautifulSoup(driver.page_source, 'html.parser')
            linha_arquivos = soup_atualizado.find('img', id=expand_link['id']).find_parent('tr').find_next_sibling('tr')
            
            if not linha_arquivos:
                continue
            
            # Procura todos os links de arquivos
            links_arquivos = linha_arquivos.find_all('a', href=True)
            
            for link in links_arquivos:
                href = link['href']
                nome_arquivo = link.get('title', 'arquivo')
                
                # Verifica se é PDF ou HTML
                if '.pdf' in nome_arquivo.lower() or 'pdf' in href.lower():
                    # É PDF - faz download
                    logging.info(f"📄 Baixando PDF: {nome_arquivo}")
                    try:
                        # Registra arquivos antes do download
                        arquivos_antes = set(os.listdir(PASTA_DOWNLOADS))
                        
                        # Clica no link
                        driver.find_element(By.XPATH, f"//a[@href='{href}']").click()
                        
                        # Aguarda download iniciar (até 60s)
                        tempo_espera = 0
                        arquivo_baixado = None
                        while tempo_espera < 60:
                            time.sleep(1)
                            tempo_espera += 1
                            arquivos_depois = set(os.listdir(PASTA_DOWNLOADS))
                            novos_arquivos = arquivos_depois - arquivos_antes
                            if novos_arquivos:
                                arquivo_baixado = novos_arquivos.pop()
                                break
                        
                        if not arquivo_baixado:
                            logging.warning(f"⚠️ Download não iniciou em 60s")
                            continue
                        
                        # Aguarda download terminar (.crdownload desaparecer)
                        caminho_arquivo = os.path.join(PASTA_DOWNLOADS, arquivo_baixado)
                        while arquivo_baixado.endswith('.crdownload'):
                            time.sleep(1)
                            if not os.path.exists(caminho_arquivo):
                                # Arquivo foi renomeado (download concluiu)
                                break
                        
                        pdfs_baixados += 1
                        logging.info(f"✅ PDF baixado com sucesso")
                        
                    except Exception as e:
                        logging.warning(f"⚠️ Erro ao baixar PDF: {e}")
                
                elif 'BuscaProcesso?PaginaAtual=6' in href:
                    # É HTML (decisão) - verifica se já foi coletada na fase pública
                    if mov_id_projudi not in decisoes_html:
                        logging.info(f"📝 Extraindo HTML (decisão) nova: mov {mov_id_projudi}")
                        try:
                            driver.execute_script(f"window.open('{href}', '_blank');")
                            time.sleep(1)  # Reduzido de 2s para 1s
                            
                            nova_aba = [aba for aba in driver.window_handles if aba != aba_principal][0]
                            driver.switch_to.window(nova_aba)
                            
                            soup_decisao = BeautifulSoup(driver.page_source, 'html.parser')
                            texto_decisao = soup_decisao.get_text(separator='\n', strip=True)
                            decisoes_html[mov_id_projudi] = texto_decisao
                            htmls_novos += 1
                            
                            driver.close()
                            driver.switch_to.window(aba_principal)
                            logging.info(f"✅ Decisão HTML nova extraída e salva")
                        except Exception as e:
                            logging.error(f"❌ Erro ao extrair HTML: {e}")
                            if len(driver.window_handles) > 1:
                                driver.switch_to.window(aba_principal)
                    else:
                        logging.info(f"⏭️ Decisão já coletada na fase pública: mov {mov_id_projudi}")
            
            # Recolhe a movimentação
            ActionChains(driver).move_to_element(
                driver.find_element(By.ID, expand_link['id'])
            ).click().perform()
            time.sleep(0.3)  # Reduzido de 0.5s para 0.3s
            
        except Exception as e:
            logging.warning(f"⚠️ Erro na movimentação {idx}: {e}")
            continue
    
    logging.info(f"✅ Processamento concluído: {pdfs_baixados} PDFs baixados, {htmls_novos} HTMLs novos extraídos")
    logging.info(f"📊 Total de decisões: {len(decisoes_html)} ({len(decisoes_publicas)} públicas + {htmls_novos} novas)")
    return movimentacoes, decisoes_html

# ==============================================================================
# FUNÇÕES DE NAVEGAÇÃO E LOGIN
# ==============================================================================

def configurar_driver():
    """Configura o Chrome com opções de download"""
    chrome_options = Options()
    #chrome_options.add_argument("--headless")  # Desativado para ver o Chrome em ação
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--remote-debugging-port=9222")
    chrome_options.add_argument("--disable-software-rasterizer")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    
    os.makedirs(PASTA_DOWNLOADS, exist_ok=True)
    
    prefs = {
        "download.default_directory": PASTA_DOWNLOADS,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "plugins.always_open_pdf_externally": True,
        "plugins.plugins_disabled": ["Chrome PDF Viewer"],
        "download.extensions_to_open": "",
        "profile.default_content_settings.popups": 0,
        "profile.default_content_setting_values.automatic_downloads": 1
    }
    chrome_options.add_experimental_option("prefs", prefs)
    
    # Usa chromedriver do sistema
    service = Service('/usr/local/bin/chromedriver')
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

def realizar_login(driver, credencial):
    """Faz login no PROJUDI"""
    usuario, senha = credencial['usuario'], credencial['senha']
    logging.info(f"🔐 Login com usuário: {usuario[-4:]}")
    
    try:
        driver.get("https://projudi.tjgo.jus.br/LogOn?PaginaAtual=-200")
        wait = WebDriverWait(driver, 20)
        
        wait.until(EC.presence_of_element_located((By.ID, "login"))).send_keys(usuario)
        driver.find_element(By.ID, "senha").send_keys(senha)
        driver.find_element(By.NAME, "entrar").click()
        wait.until(EC.frame_to_be_available_and_switch_to_it((By.NAME, "userMainFrame")))
        
        logging.info(f"✅ Login realizado com sucesso")
        return True
    except Exception as e:
        logging.error(f"❌ Falha no login: {e}")
        return False

def solicitar_acesso_processo(driver, wait, numero_processo):
    """
    Solicita acesso ao processo
    Retorna: ('sucesso', True) | ('rate_limit', False) | ('erro', False)
    """
    logging.info(f"🔓 Solicitando acesso ao processo...")
    
    try:
        driver.get("https://projudi.tjgo.jus.br/DescartarPendenciaProcesso?PaginaAtual=8")
        time.sleep(2)
        
        try:
            dialogo = WebDriverWait(driver, 7).until(EC.visibility_of_element_located((By.ID, "dialog")))
            botao_ok = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'OK')]")))
            botao_ok.click()
            logging.info(f"✅ Acesso concedido!")
            
            # Volta para o processo
            driver.get("https://projudi.tjgo.jus.br/BuscaProcesso?PaginaAtual=4")
            campo_busca = wait.until(EC.visibility_of_element_located((By.ID, "ProcessoNumero")))
            campo_busca.clear()
            campo_busca.send_keys(numero_processo)
            try:
                driver.find_element(By.NAME, "imaLimparProcessoStatus").click()
            except:
                pass
            driver.find_element(By.NAME, "imgSubmeter").click()
            time.sleep(2)
            logging.info(f"✅ De volta ao processo")
            
            return ('sucesso', True)
            
        except TimeoutException:
            src = driver.page_source
            if "Usuário tem que esperar 24h" in src or "usuário já tem acesso" in src:
                logging.info(f"✅ Acesso já existente")
                
                # Volta para o processo
                driver.get("https://projudi.tjgo.jus.br/BuscaProcesso?PaginaAtual=4")
                campo_busca = wait.until(EC.visibility_of_element_located((By.ID, "ProcessoNumero")))
                campo_busca.clear()
                campo_busca.send_keys(numero_processo)
                try:
                    driver.find_element(By.NAME, "imaLimparProcessoStatus").click()
                except:
                    pass
                driver.find_element(By.NAME, "imgSubmeter").click()
                time.sleep(2)
                logging.info(f"✅ De volta ao processo")
                
                return ('sucesso', True)
                
            elif "Só é permitido" in src or "atingiu o limite" in src:
                logging.warning(f"🚫 RATE LIMIT atingido! Precisa reiniciar navegador.")
                return ('rate_limit', False)
            else:
                logging.error(f"❌ Estado desconhecido ao solicitar acesso")
                return ('erro', False)
                
    except Exception as e:
        logging.error(f"❌ Erro ao solicitar acesso: {e}")
        return ('erro', False)

# ==============================================================================
# FUNÇÃO PRINCIPAL DE PROCESSAMENTO
# ==============================================================================

def verificar_filtro_antes_acesso(driver, numero_processo):
    """
    Verifica se o processo passa no filtro ANTES de pedir acesso
    Retorna: True se passa no filtro, False se não passa
    """
    if not CONFIG["palavras_chave"] and not CONFIG["palavras_nega"]:
        return True  # Sem filtros, aceita tudo
    
    try:
        logging.info(f"🔍 Verificando filtro antes de pedir acesso...")
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        linhas_mov = soup.find_all('tr', class_='filtro-entrada')
        
        # Verifica se alguma movimentação passa no filtro
        for mov_tr in linhas_mov:
            colunas = mov_tr.find_all('td')
            if len(colunas) >= 2:
                descricao = colunas[1].get_text(separator=' ', strip=True)
                if filtrar_movimentacao(descricao):
                    logging.info(f"✅ Processo tem movimentações que passam no filtro")
                    return True
        
        logging.info(f"⏭️ Nenhuma movimentação passa no filtro. Pulando processo.")
        return False
        
    except Exception as e:
        logging.warning(f"⚠️ Erro ao verificar filtro: {e}. Processando por segurança.")
        return True  # Em caso de erro, processa

def processar_processo(driver, wait, numero_processo, devedor):
    """
    Processa um processo completo
    Retorna: ('sucesso', True) | ('rate_limit', False) | ('filtrado', False) | ('erro', False)
    """
    logging.info(f"\n{'='*80}")
    logging.info(f"🔍 PROCESSO: {numero_processo} - {devedor}")
    logging.info(f"{'='*80}\n")
    
    try:
        # Busca o processo
        driver.get("https://projudi.tjgo.jus.br/BuscaProcesso?PaginaAtual=4")
        campo_busca = wait.until(EC.visibility_of_element_located((By.ID, "ProcessoNumero")))
        campo_busca.clear()
        campo_busca.send_keys(numero_processo)
        try:
            driver.find_element(By.NAME, "imaLimparProcessoStatus").click()
        except:
            pass
        driver.find_element(By.NAME, "imgSubmeter").click()
        time.sleep(2)
        
        # COLETA DECISÕES PÚBLICAS (ANTES DE PEDIR ACESSO)
        decisoes_publicas = extrair_decisoes_publicas(driver, numero_processo)
        
        # VERIFICA FILTRO ANTES DE PEDIR ACESSO
        if not verificar_filtro_antes_acesso(driver, numero_processo):
            logging.info(f"⏭️ Processo não passa no filtro. Pulando sem pedir acesso.")
            logging.info(f"📊 Decisões públicas coletadas: {len(decisoes_publicas)}")
            return ('filtrado', False)
        
        # Solicita acesso
        status, sucesso = solicitar_acesso_processo(driver, wait, numero_processo)
        if not sucesso:
            if status == 'rate_limit':
                logging.error(f"🚫 Rate limit atingido")
                return ('rate_limit', False)
            else:
                logging.error(f"❌ Não foi possível obter acesso ao processo")
                return ('erro', False)
        
        # Extrai dados gerais
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        dados_gerais = extrair_dados_gerais(soup)
        
        # Processa movimentações e arquivos (passa as decisões públicas já coletadas)
        movimentacoes, decisoes_html = processar_movimentacoes_e_arquivos(driver, wait, numero_processo, decisoes_publicas)
        
        # Salva no banco
        salvar_dados_completos(numero_processo, dados_gerais, movimentacoes, decisoes_html)
        
        return ('sucesso', True)
        
    except Exception as e:
        logging.error(f"❌ Erro crítico no processo {numero_processo}: {e}")
        import traceback
        traceback.print_exc()
        
        # Salva log completo do erro
        try:
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            erro_dir = f"logs_erro/{numero_processo}_{timestamp}"
            os.makedirs(erro_dir, exist_ok=True)
            
            # Salva screenshot
            screenshot_path = f"{erro_dir}/screenshot_erro.png"
            driver.save_screenshot(screenshot_path)
            logging.info(f"📸 Screenshot salvo em: {screenshot_path}")
            
            # Salva HTML da página
            html_path = f"{erro_dir}/pagina_erro.html"
            with open(html_path, 'w', encoding='utf-8') as f:
                f.write(driver.page_source)
            logging.info(f"📄 HTML salvo em: {html_path}")
            
            # Salva traceback completo
            traceback_path = f"{erro_dir}/traceback.txt"
            with open(traceback_path, 'w', encoding='utf-8') as f:
                f.write(f"Processo: {numero_processo}\n")
                f.write(f"Erro: {str(e)}\n\n")
                f.write(traceback.format_exc())
            logging.info(f"📝 Traceback salvo em: {traceback_path}")
            
            logging.info(f"✅ Logs de erro salvos em: {erro_dir}")
        except Exception as log_err:
            logging.error(f"⚠️ Não foi possível salvar logs de erro: {log_err}")
        
        return ('erro', False)

# ==============================================================================
# MAIN
# ==============================================================================

def main():
    print(f"\n{Style.BRIGHT}{'='*80}")
    print(f"{Fore.CYAN}🤖 ROBÔ PROJUDI - COMPLETO v2.0")
    print(f"{'='*80}{Style.RESET_ALL}\n")
    
    # Adiciona colunas de status se necessário
    adicionar_coluna_status_se_necessario()
    
    # Verifica se existe arquivo CSV e pergunta se quer usar
    usar_csv = False
    csv_existe = os.path.exists(ARQUIVO_PROCESSOS_CSV)
    
    if csv_existe:
        print(f"{Fore.YELLOW}📄 Arquivo CSV detectado: {ARQUIVO_PROCESSOS_CSV}{Style.RESET_ALL}")
        
        while True:
            resposta = input(f"\n{Fore.YELLOW}Deseja usar o arquivo CSV? (s/n): {Style.RESET_ALL}").strip().lower()
            if resposta in ['s', 'sim', 'y', 'yes']:
                usar_csv = True
                print(f"{Fore.GREEN}✅ Usando arquivo CSV{Style.RESET_ALL}\n")
                break
            elif resposta in ['n', 'nao', 'não', 'no']:
                print(f"{Fore.GREEN}✅ Usando Banco de Dados{Style.RESET_ALL}\n")
                break
            else:
                print(f"{Fore.RED}Resposta inválida! Digite 's' para sim ou 'n' para não.{Style.RESET_ALL}")
    
    # Conta processos disponíveis
    if usar_csv:
        try:
            df_csv = pd.read_csv(ARQUIVO_PROCESSOS_CSV)
            total_pendentes = len(df_csv)
            print(f"{Fore.CYAN}📊 Total de processos no CSV: {total_pendentes}{Style.RESET_ALL}")
        except Exception as e:
            print(f"{Fore.RED}❌ Erro ao ler CSV: {e}{Style.RESET_ALL}")
            return
    else:
        with sqlite3.connect(BANCO_ESTRATEGICO) as conn:
            cursor = conn.cursor()
            cursor.execute("SELECT COUNT(*) FROM ExtracaoDiario WHERE status_coleta IS NULL OR status_coleta IN ('pendente', 'erro')")
            total_pendentes = cursor.fetchone()[0]
        print(f"{Fore.CYAN}📊 Total de processos pendentes no DB: {total_pendentes}{Style.RESET_ALL}")
    
    # Pergunta sobre filtros de palavras-chave
    print(f"\n{Fore.YELLOW}{'='*80}")
    print(f"CONFIGURAÇÃO DE FILTROS DE PALAVRAS-CHAVE")
    print(f"{'='*80}{Style.RESET_ALL}")
    print(f"\n{Fore.CYAN}Você pode filtrar movimentações por palavras-chave na DESCRIÇÃO:")
    print(f"  • Palavras POSITIVAS: descrição precisa conter pelo menos uma")
    print(f"  • Palavras NEGATIVAS: descrição não pode conter nenhuma")
    print(f"  • Formato: separe por vírgula (ex: cpc,cálculo,honorários)")
    print(f"  • Deixe em branco para não filtrar")
    print(f"  • Filtro se aplica à descrição da movimentação, não ao texto da decisão{Style.RESET_ALL}\n")
    
    palavras_pos = input(f"{Fore.GREEN}Digite palavras-chave POSITIVAS (ou Enter para pular): {Style.RESET_ALL}").strip()
    if palavras_pos:
        CONFIG["palavras_chave"] = [p.strip() for p in palavras_pos.split(',') if p.strip()]
        print(f"{Fore.GREEN}✅ Palavras positivas: {CONFIG['palavras_chave']}{Style.RESET_ALL}")
    
    palavras_neg = input(f"{Fore.RED}Digite palavras-chave NEGATIVAS (ou Enter para pular): {Style.RESET_ALL}").strip()
    if palavras_neg:
        CONFIG["palavras_nega"] = [p.strip() for p in palavras_neg.split(',') if p.strip()]
        print(f"{Fore.RED}✅ Palavras negativas: {CONFIG['palavras_nega']}{Style.RESET_ALL}")
    
    if not CONFIG["palavras_chave"] and not CONFIG["palavras_nega"]:
        print(f"{Fore.YELLOW}⚠️ Nenhum filtro configurado. Todas as movimentações serão processadas.{Style.RESET_ALL}")
    
    # Pergunta sobre o tempo de espera do rate limit
    while True:
        try:
            tempo_espera_input = input(f"\n{Fore.YELLOW}Quantos segundos aguardar em caso de 'rate limit'? (Padrão: 7): {Style.RESET_ALL}").strip()
            if not tempo_espera_input:
                tempo_espera_rate_limit = 7
                break
            tempo_espera_rate_limit = int(tempo_espera_input)
            if tempo_espera_rate_limit > 0:
                break
            else:
                print(f"{Fore.RED}Digite um número positivo!")
        except ValueError:
            print(f"{Fore.RED}Digite um número válido!")
    print(f"{Fore.GREEN}✅ Tempo de espera configurado: {tempo_espera_rate_limit} segundos{Style.RESET_ALL}")
    
    # Pergunta quantos processos processar
    while True:
        try:
            qtd = input(f"\n{Fore.YELLOW}Quantos processar? (1-{total_pendentes} ou 0 para todos): {Style.RESET_ALL}")
            qtd = int(qtd)
            if qtd == 0:
                qtd = total_pendentes
                break
            elif 1 <= qtd <= total_pendentes:
                break
            else:
                print(f"{Fore.RED}Número inválido!")
        except ValueError:
            print(f"{Fore.RED}Digite um número válido!")
    
    # Busca processos da fonte escolhida
    processos_selecionados = []
    
    if usar_csv:
        # Lê do CSV
        df_processos = pd.read_csv(ARQUIVO_PROCESSOS_CSV).head(qtd)
        for idx, row in df_processos.iterrows():
            processos_selecionados.append({
                "id": None,  # CSV não tem ID do banco
                "numero": formatar_numero_cnj(str(row['numero_processo'])),
                "devedor": str(row.get('devedor', 'N/A'))
            })
    else:
        # Busca do banco
        processos_db = buscar_processos_do_banco(qtd)
        for processo_id, numero_cnj in processos_db:
            processos_selecionados.append({
                "id": processo_id,
                "numero": formatar_numero_cnj(numero_cnj),
                "devedor": "N/A"
            })
    
    if usar_csv:
        print(f"{Fore.GREEN}✅ {len(processos_selecionados)} processo(s) carregado(s) do CSV!\n")
    else:
        print(f"{Fore.GREEN}✅ {len(processos_selecionados)} processo(s) carregado(s) do banco de dados!\n")
    
    # Pergunta qual credencial usar
    print(f"\n{Fore.YELLOW}{'='*80}")
    print(f"SELEÇÃO DE CREDENCIAL")
    print(f"{'='*80}{Style.RESET_ALL}")
    for i, cred in enumerate(POOL_DE_CREDENCIAIS, 1):
        print(f"  [{i}] - Usuário: {cred['usuario']}")
    
    while True:
        try:
            escolha = int(input(f"\n{Fore.YELLOW}Escolha a credencial (1-{len(POOL_DE_CREDENCIAIS)}): {Style.RESET_ALL}"))
            if 1 <= escolha <= len(POOL_DE_CREDENCIAIS):
                credencial = POOL_DE_CREDENCIAIS[escolha - 1]
                print(f"{Fore.GREEN}✅ Usando credencial: {credencial['usuario']}{Style.RESET_ALL}\n")
                break
            else:
                print(f"{Fore.RED}Opção inválida!")
        except ValueError:
            print(f"{Fore.RED}Digite um número válido!")
    
    # Configura driver e faz login
    driver = configurar_driver()
    wait = WebDriverWait(driver, 20)
    
    if not realizar_login(driver, credencial):
        print(f"{Fore.RED}❌ Falha no login. Encerrando.")
        return
    
    # Processa cada processo
    sucessos = 0
    filtrados = 0
    erros = 0
    
    for idx, processo in enumerate(processos_selecionados, 1):
        print(f"\n{Fore.CYAN}▶️  Processo {idx}/{qtd}")
        
        # Marca como coletando (somente se vier do banco)
        if processo['id'] is not None:
            atualizar_status_processo(processo['id'], 'coletando')
        
        status, resultado = processar_processo(driver, wait, processo['numero'], processo['devedor'])
        
        if status == 'sucesso':
            if processo['id'] is not None:
                atualizar_status_processo(processo['id'], 'sucesso')
            sucessos += 1
        elif status == 'filtrado':
            if processo['id'] is not None:
                atualizar_status_processo(processo['id'], 'filtrado')
            filtrados += 1
        elif status == 'rate_limit':
            # REINICIA NAVEGADOR PARA ACESSO ILIMITADO
            logging.warning(f"\n{'='*80}")
            logging.warning(f"🔄 RATE LIMIT ATINGIDO! Reiniciando navegador...")
            logging.warning(f"{'='*80}\n")
            
            try:
                driver.quit()
                time.sleep(tempo_espera_rate_limit)
            except:
                pass
            
            # Abre novo navegador e faz login
            driver = configurar_driver()
            wait = WebDriverWait(driver, 20)
            
            if not realizar_login(driver, credencial):
                logging.error(f"❌ Falha no login após reiniciar. Encerrando.")
                break
            
            logging.info(f"✅ Navegador reiniciado e login realizado!")
            logging.info(f"🔄 Tentando processar novamente: {processo['numero']}\n")
            
            # Tenta processar novamente
            status, resultado = processar_processo(driver, wait, processo['numero'], processo['devedor'])
            if status == 'sucesso':
                if processo['id'] is not None:
                    atualizar_status_processo(processo['id'], 'sucesso')
                sucessos += 1
            elif status == 'filtrado':
                if processo['id'] is not None:
                    atualizar_status_processo(processo['id'], 'filtrado')
                filtrados += 1
            else:
                if processo['id'] is not None:
                    atualizar_status_processo(processo['id'], 'erro', str(resultado))
                erros += 1
        else:
            if processo['id'] is not None:
                atualizar_status_processo(processo['id'], 'erro', str(resultado))
            erros += 1
    
    # Resultado final
    print(f"\n{Style.BRIGHT}{'='*80}")
    print(f"{Fore.GREEN}✅ CONCLUÍDO!")
    print(f"  • Sucessos: {sucessos}/{qtd}")
    print(f"  • Filtrados: {filtrados}/{qtd}")
    print(f"  • Erros: {erros}/{qtd}")
    print(f"{'='*80}{Style.RESET_ALL}\n")
    
    # Mantém navegador aberto
    print(f"{Fore.CYAN}Navegador permanecerá aberto. Pressione Ctrl+C para fechar.")
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print(f"\n{Fore.YELLOW}Fechando navegador...")
        driver.quit()
        print(f"{Fore.GREEN}Navegador fechado!")

if __name__ == "__main__":
    main()